<a href="https://colab.research.google.com/github/kjinb1212/Falldown-detection-KD/blob/main/teacher_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.model_selection import train_test_split

from glob import glob
import os
import sys
from PIL import Image
import timm

from efficientnet_pytorch import EfficientNet
import pandas as pd
from torchsummary import summary


In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, input_size, train = True, padding = True, normalize = False,
                 bright_ness = 0.2, hue = 0.15, contrast = 0.15, random_Hflip = 0.3, rotate_deg = 20):
        orig_normal_path = glob(os.path.join(root_dir, 'normal') + '/*.jpg')
        orig_fall_path = glob(os.path.join(root_dir, 'falldown') + '/*.jpg')
        orig_back_path = glob(os.path.join(root_dir, 'background') + '/*.jpg')
        
        normal_paths = []
        fall_paths = []
        back_paths = []
        
        for path in orig_normal_path:
            img = Image.open(path)
            if min(img.size[0], img.size[1]) < 32:
                pass
            else:
                normal_paths.append(path)
                
        for path in orig_fall_path:
            img = Image.open(path)
            if min(img.size[0], img.size[1]) < 32:
                pass
            else:
                fall_paths.append(path)
        
        for path in orig_back_path:
            img = Image.open(path)
            if min(img.size[0], img.size[1]) < 32:
                pass
            else:
                back_paths.append(path)
                        
        self.total_paths = normal_paths + fall_paths + back_paths
        self.labels = [0] * len(normal_paths) + [1] * len(fall_paths) + [2] * len(back_paths)
        
        transform = []
        if train:
            #transform.append(torchvision.transforms.ColorJitter(brightness=bright_ness, hue=hue, contrast=contrast))
            transform.append(torchvision.transforms.RandomHorizontalFlip(p=random_Hflip))
            #transform.append(torchvision.transforms.RandomCrop(224))
            transform.append(torchvision.transforms.RandomRotation(degrees=rotate_deg))
        transform.append(torchvision.transforms.ToTensor())
        if normalize:
            transform.append(torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]))
        if padding:
            transform.append(lambda x: torchvision.transforms.Pad(((128 - x.shape[2]) // 2, (128 - x.shape[1]) // 2), fill=0,
                                                     padding_mode="constant")(x))
        transform.append(torchvision.transforms.Resize((input_size, input_size)))
        self.transform = torchvision.transforms.Compose(transform)
        
        
    def __len__(self):
        return len(self.total_paths)

    def __getitem__(self, index):
        img = Image.open(self.total_paths[index])
        img = self.transform(img)
        return img, self.labels[index]

In [ ]:
def train(model, train_loader, test_loader, criterion, optimizer, epochs, save_name):
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, patience=4, verbose=True)
    best_loss = None
    best_acc = None
    patience = 0 
    
    history = {'loss': [], 'acc': []}
    
    for epoch in range(epochs):
        print("--------- epoch : {} ------------".format(epoch+1))

        model.train()
        train_losses = []
        for data, label in train_loader:
            data = data.to(device)
            label = label.to(device)
            
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, label)
            train_losses.append(loss.item())
            loss.backward()
            optimizer.step()
            torch.cuda.empty_cache()
        train_loss = np.average(train_losses)
        print("train loss: {}".format(train_loss))
        
        model.eval()
        test_losses = []
        correct = 0
        total = 0
        with torch.no_grad():
            for data, label in test_loader:
                data = data.to(device)
                label = label.to(device)
                
                output = model(data)
                loss = criterion(output, label)
                test_losses.append(loss.item())
                _, predict = torch.max(output.data, 1)
                correct += (predict == label).sum().item()
                total += label.size(0)
            test_loss = np.average(test_losses)
            test_acc = 100 * correct / total
            #if (epoch + 1) % 5 ==0:
            print("test loss: {}, \ttest acc: {}%".format(test_loss, test_acc))

            history['loss'].append(test_loss)
            history['acc'].append(test_acc)
            
            if (best_loss is None) or (best_loss > test_loss):
                best_loss = test_loss
                best_acc = test_acc
                torch.save(model.state_dict(), 'new_model_weights/' + save_name + '.pth')
                print('epoch: {}\t Best loss: {}'.format(epoch + 1, best_loss))
                patience = 0
            else:
                patience += 1

            if patience > 7:
                print("early stop at {} epoch".format(epoch + 1))
                break

            scheduler.step(metrics=test_loss)
    
    print("best loss: {},\t best acc: {}%\n\n".format(best_loss, best_acc))
    return best_loss, best_acc, history 

In [ ]:
INPUT_SIZE = 128
PADDING = False
NORMALIZE = False
BATCHSIZE = 128
NUMEPOCH = 100

train_data = CustomDataset(
    root_dir='train',
    input_size=INPUT_SIZE, train=True, padding=PADDING, normalize=NORMALIZE,
    bright_ness=0, hue=01.5, contrast=0.15, random_Hflip=0, rotate_deg=0)

test_data = CustomDataset(
    root_dir='validation',
    input_size=INPUT_SIZE, train=False, padding=PADDING, normalize=NORMALIZE,
    bright_ness=0, hue=01.5, contrast=0.15, random_Hflip=0, rotate_deg=0)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCHSIZE, shuffle=True, num_workers=32, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCHSIZE, shuffle=False, num_workers=32, drop_last=True)

# train every model together

In [ ]:
models = ['efficientnet-b0', 'efficientnet-b1', 'efficientnet-b2', 'efficientnet-b3', 'efficientnet-b4']
logs = []
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

for model_name in models:
    torch.cuda.empty_cache()
    teacher_model = EfficientNet.from_pretrained(model_name, num_classes=3).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(teacher_model.parameters(), weight_decay=1e-4, lr=0.0001)
    model_name = 'this_is_test'
    
    loss, acc, history = train(model=teacher_model, train_loader=train_loader, test_loader=test_loader,
          criterion = criterion, optimizer=optimizer, epochs=NUMEPOCH, save_name=model_name) 
    
    s = 'model: ' + model_name + '\t loss: {}\t acc: {}'.format(loss, acc)
    logs.append(s)
    
    df = pd.DataFrame(history)
    df.to_csv("new_history/"+ model_name+ "_history.csv", mode='w')
    

Loaded pretrained weights for efficientnet-b0
--------- epoch : 1 ------------
train loss: 0.3353194579043809
test loss: 0.3699789047241211, 	test acc: 84.6875%
epoch: 1	 Best loss: 0.3699789047241211
--------- epoch : 2 ------------


KeyboardInterrupt: 

In [ ]:
for log in logs:
    print(log)

model: efficientnet-b0	 loss: 0.2674218351021409	 acc: 89.990234375
model: efficientnet-b1	 loss: 0.28221857687458396	 acc: 88.525390625
model: efficientnet-b2	 loss: 0.2607212816365063	 acc: 88.916015625
model: efficientnet-b3	 loss: 0.2985588605515659	 acc: 88.720703125
model: efficientnet-b4	 loss: 0.27449027681723237	 acc: 89.697265625


# voting classifier inference

In [ ]:
def get_teacher_output(model, loader):
    model.eval()
    output = []
    with torch.no_grad():
        for data, _ in loader:
            data = data.to(device)
            output.append(model(data))
    torch.cuda.empty_cache()
    return output

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


ensamble_list = [['b0', 'b1'], ['b0', 'b2'], ['b0', 'b3'], ['b0', 'b4'], ['b1', 'b2'], ['b1', 'b3'], ['b1', 'b4'], ['b2', 'b3'], ['b2', 'b4'], ['b3', 'b4'],
                ['b0', 'b1', 'b2'], ['b0', 'b1', 'b3'], ['b0', 'b1', 'b4'], ['b0', 'b2', 'b3'], ['b0', 'b2', 'b4'], ['b0', 'b3', 'b4'], ['b1', 'b2', 'b3'],
                ['b1', 'b2', 'b4'], ['b2', 'b3', 'b4'], ['b0', 'b1', 'b2', 'b3'], ['b0', 'b1', 'b2', 'b4'], ['b0', 'b1', 'b3', 'b4'], ['b0', 'b2', 'b3', 'b4'], 
                ['b1', 'b2', 'b3', 'b4'], ['b0', 'b1', 'b2', 'b3', 'b4']]

logs = []
for teachers in ensamble_list:
    teacher_output = []
    for teacher_name in teachers:
        model_name = 'efficientnet-' + teacher_name
        torch.cuda.empty_cache()

        teacher_model = EfficientNet.from_pretrained(model_name, num_classes=3).to(device)
        teacher_model.load_state_dict(torch.load('new_model_weights/'+ model_name + '.pth'))

        teacher_output.append(get_teacher_output(teacher_model, test_loader))

    criterion = nn.CrossEntropyLoss()
    n_teacher = len(teachers)
    test_losses = []
    correct = 0
    total = 0
    for i, (data, label) in enumerate(test_loader):
        label = label.to(device)
        output = teacher_output[0][i]
        for j in range(1, n_teacher):
            output += teacher_output[j][i]
        output /= n_teacher
        loss = criterion(output, label)
        test_losses.append(loss.item())
        _, predict = torch.max(output.data, 1)
        correct += (predict == label).sum().item()
        total += label.size(0)
    test_loss = np.average(test_losses)
    test_acc = 100 * correct / total

    save_name = 'voting'
    for teacher_name in teachers:
        save_name += '-' + teacher_name
    s = save_name + "\t loss: {}\t acc: {}%".format(test_loss, test_acc)
    logs.append(s)

Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b1
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b3
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b4
Loaded pretrained weights for efficientnet-b1
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b1
Loaded pretrained weights for efficientnet-b3
Loaded pretrained weights for efficientnet-b1
Loaded pretrained weights for efficientnet-b4
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b3
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b4
Loaded pretrained weights for efficientnet-b3
Loaded pretrained weights for efficientnet-b4
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for effi

In [ ]:
for log in logs:
    print(log)

voting-b0-b1	 loss: 0.2546918373554945	 acc: 89.94140625%
voting-b0-b2	 loss: 0.24643599847331643	 acc: 89.84375%
voting-b0-b3	 loss: 0.26559690991416574	 acc: 89.6484375%
voting-b0-b4	 loss: 0.2490836069919169	 acc: 90.380859375%
voting-b1-b2	 loss: 0.2578577911481261	 acc: 88.818359375%
voting-b1-b3	 loss: 0.2762367641553283	 acc: 88.8671875%
voting-b1-b4	 loss: 0.26070934953168035	 acc: 89.306640625%
voting-b2-b3	 loss: 0.2659572479315102	 acc: 88.720703125%
voting-b2-b4	 loss: 0.25041161896660924	 acc: 89.6484375%
voting-b3-b4	 loss: 0.2670891396701336	 acc: 89.501953125%
voting-b0-b1-b2	 loss: 0.24713713163509965	 acc: 89.35546875%
voting-b0-b1-b3	 loss: 0.25977719109505415	 acc: 89.55078125%
voting-b0-b1-b4	 loss: 0.24803350819274783	 acc: 89.892578125%
voting-b0-b2-b3	 loss: 0.25383210834115744	 acc: 89.794921875%
voting-b0-b2-b4	 loss: 0.2422612914815545	 acc: 90.234375%
voting-b0-b3-b4	 loss: 0.25434386311098933	 acc: 89.892578125%
voting-b1-b2-b3	 loss: 0.2622634903527796	 ac

# teacher models summary

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

torch.cuda.empty_cache()
teacher_model = EfficientNet.from_pretrained('efficientnet-b4', num_classes=3).to(device)
teacher_model.load_state_dict(torch.load('new_model_weights/'+ 'efficientnet-b4' + '.pth'))
summary(teacher_model, (3, 128, 128))


Loaded pretrained weights for efficientnet-b4
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 129, 129]               0
Conv2dStaticSamePadding-2           [-1, 48, 64, 64]           1,296
       BatchNorm2d-3           [-1, 48, 64, 64]              96
MemoryEfficientSwish-4           [-1, 48, 64, 64]               0
         ZeroPad2d-5           [-1, 48, 66, 66]               0
Conv2dStaticSamePadding-6           [-1, 48, 64, 64]             432
       BatchNorm2d-7           [-1, 48, 64, 64]              96
MemoryEfficientSwish-8           [-1, 48, 64, 64]               0
          Identity-9             [-1, 48, 1, 1]               0
Conv2dStaticSamePadding-10             [-1, 12, 1, 1]             588
MemoryEfficientSwish-11             [-1, 12, 1, 1]               0
         Identity-12             [-1, 12, 1, 1]               0
Conv2dStaticSamePadding-13        